In [1]:
import numpy as np

from anony_process_pipeline import *
from src.modules.utils_image import pillow_image_to_bytes, numpy_image_to_pillow
from database_controller import DatabaseController


Configuration chargée et vérifiée.


In [2]:
from werkzeug.datastructures import FileStorage
import os

def import_subject_images(ids):
    image_folder = "../../data/yalefaces"
    sujet = f"subject0{ids}_"
    image_files = []
    for filename in os.listdir(image_folder):
        if filename.startswith(sujet):
            chemin_fichier = f"{image_folder}/{filename}"
            file = open(chemin_fichier, "rb")
            file_storage = FileStorage(stream=file, filename=filename)
            image_files.append(file_storage)
    return image_files

# Import images
img_list = import_subject_images(3)

# Print result
img_list

[<FileStorage: 'subject03_centerlight.png' (None)>,
 <FileStorage: 'subject03_glasses.png' (None)>,
 <FileStorage: 'subject03_happy.png' (None)>,
 <FileStorage: 'subject03_leftlight.png' (None)>,
 <FileStorage: 'subject03_noglasses.png' (None)>,
 <FileStorage: 'subject03_normal.png' (None)>,
 <FileStorage: 'subject03_rightlight.png' (None)>,
 <FileStorage: 'subject03_sad.png' (None)>,
 <FileStorage: 'subject03_sleepy.png' (None)>,
 <FileStorage: 'subject03_surprised.png' (None)>,
 <FileStorage: 'subject03_wink.png' (None)>]

In [3]:
# ---------------------------------------
# Étape 1 : Preprocessing (Simplifiée)
# ---------------------------------------
image_size = (100, 100)
res1, image_size = run_preprocessing(filestorage_list=img_list, image_size_override=image_size)

# IMG IMPORT ENDING: Realise images in storage ###
for img_in_storage in img_list: ##################
    img_in_storage.close() #######################
##################################################

# Print result
res1

2025-04-08 10:34:32,482 - INFO - Exécution du pré-traitement standard avec IMAGE_SIZE=(100, 100)...
2025-04-08 10:34:32,483 - INFO - Traitement de 11 fichiers FileStorage.
Preprocessing (FileStorage List): 100%|██████████| 11/11 [00:00<00:00, 654.24it/s]
2025-04-08 10:34:32,503 - INFO - Pré-traitement terminé. 11 images traitées pour 1 sujets.


{'upload_subject_1': [{'resized_image': <PIL.Image.Image image mode=RGB size=100x100>,
   'grayscale_image': <PIL.Image.Image image mode=L size=100x100>,
   'normalized_image': array([[0.81960784, 0.81960784, 0.81960784, ..., 1.        , 1.        ,
           1.        ],
          [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
           1.        ],
          [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
           1.        ],
          ...,
          [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
           0.98823529],
          [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
           1.        ],
          [0.73333333, 0.73333333, 0.73333333, ..., 0.73333333, 0.73333333,
           0.73333333]]),
   'imageId': 'upload_img_0'},
  {'resized_image': <PIL.Image.Image image mode=RGB size=100x100>,
   'grayscale_image': <PIL.Image.Image image mode=L size=100x100>,
   'normalized_image': array([[0.81960784, 0

In [4]:
# ---------------------------------------
# Étape 2 : K-Same-Pixel (Nouvelle Fonction Modulaire)
# ---------------------------------------
k_same_k_value = 4
res2 = run_k_same_anonymization(res1, k_same_k_value)

# Print result
res2
# [img['flattened_anonymized_image'] for img in res2["upload_subject_1"]]

2025-04-08 10:34:34,977 - INFO - Application de K-Same-Pixel avec k=4...
K-Same Pixel: 100%|██████████| 1/1 [00:00<00:00, 912.20it/s]
2025-04-08 10:34:34,981 - INFO - Traitement K-Same-Pixel terminé.


{'upload_subject_1': [{'flattened_anonymized_image': array([0.7490196 , 0.7647059 , 0.7529412 , ..., 0.5647059 , 0.5568628 ,
          0.54901963], dtype=float32),
   'imageId': 'upload_img_0'},
  {'flattened_anonymized_image': array([0.81960785, 0.81960785, 0.81960785, ..., 0.73333335, 0.73333335,
          0.73333335], dtype=float32),
   'imageId': 'upload_img_1'},
  {'flattened_anonymized_image': array([0.81960785, 0.81960785, 0.81960785, ..., 0.73333335, 0.73333335,
          0.73333335], dtype=float32),
   'imageId': 'upload_img_2'},
  {'flattened_anonymized_image': array([0.7490196 , 0.7647059 , 0.7529412 , ..., 0.5647059 , 0.5568628 ,
          0.54901963], dtype=float32),
   'imageId': 'upload_img_3'},
  {'flattened_anonymized_image': array([0.81960785, 0.81960785, 0.81960785, ..., 0.73333335, 0.73333335,
          0.73333335], dtype=float32),
   'imageId': 'upload_img_4'},
  {'flattened_anonymized_image': array([0.81960785, 0.81960785, 0.81960785, ..., 0.5882353 , 0.5882353 ,


In [5]:
# ---------------------------------------
# Étape 3 : PEEP Eigenface
# ---------------------------------------

flattened_stack_list = [img['flattened_anonymized_image'] for img in res2["upload_subject_1"]]
flattened_stack_np = np.array(flattened_stack_list, dtype=np.float32)

n_samples, n_features = flattened_stack_np.shape
n_components_ratio = 0.8
n_components = min(max(1, int(n_components_ratio * n_samples)), n_features)

res3 = run_eigenface(flattened_stack_np, n_components)

# Print result
pca, mean_face, projection = res3 # Eingenface a disparue...
print("n_components:", n_components)
mean_face

n_components: 8


array([[0.7939394 , 0.79964346, 0.7953654 , ..., 0.9868093 , 0.9853833 ,
        0.9782531 ],
       [0.9714795 , 0.9700535 , 0.9700535 , ..., 0.9843138 , 0.9853833 ,
        0.9786096 ],
       [0.9686275 , 0.96577543, 0.9729055 , ..., 0.9843138 , 0.9828877 ,
        0.9771835 ],
       ...,
       [0.9472371 , 0.9401069 , 0.9329768 , ..., 0.9001782 , 0.9012478 ,
        0.89696974],
       [0.9386809 , 0.9329768 , 0.9286988 , ..., 0.9016043 , 0.8998217 ,
        0.8987522 ],
       [0.6905527 , 0.6877006 , 0.6862745 , ..., 0.6720143 , 0.6691622 ,
        0.66631025]], dtype=float32)

In [7]:
# Étape 4 : Ajout de bruit différentiel
epsilon = 0.01
res4 = run_add_noise(projection, epsilon, sensitivity=1.0)

# Étape 5 : Reconstruction
res5 = run_reconstruction(pca, res4, image_size)

# Print result
import base64
from io import BytesIO
from PIL import Image
imgs = [Image.open(BytesIO(base64.b64decode(img))) for img in res5]
print(len(imgs))
imgs[0].show()
res5

11


['...']

In [17]:
db = DatabaseController(r"..\..\data\database.db")

user_id = db.add_user(np.array(res5))
print(f"user_id: {user_id}")

user = db.get_user(user_id)

base64_str = user[0]
image_bytes = base64.b64decode(base64_str)
image = Image.open(io.BytesIO(image_bytes))
image.show()

Create db connection at: D:\myDocs\Apprentissage\Isep\Year_3\Brno-Privacy_Preserving_Face_Recognition_Project\data\database.db
user_id: 4
